In [4]:
import requests
from shapely.geometry import LineString
import pandas as pd
import json
import osmnx as ox
import pyrosm
import folium

def find_edges(G, hwys):
    edges = []
    for u, v, k, data in G.edges(keys=True, data='highway'):
        check1 = isinstance(data, str) and data not in hwys
        check2 = isinstance(data, list) and all([d not in hwys for d in data])
        if check1 or check2:
            edges.append((u, v, k))
    return set(edges)

hwy_colors = {
        'motorway': 'purple',
        'trunk': 'pink',
        'primary': 'green',
        'secondary': 'yellow',
        'tertiary': 'orange',
        'unclassified': 'blue',
        'residential': 'red',
    }

In [5]:
# Create request URL
BASE_URL = "https://api.tomtom.com/"
API_KEY = "ThfURaOhGRaty7mx9AKALwrbyIZApHd8"
#zoom = 22
TRAFFIC_URL = "traffic/services/4/flowSegmentData/absolute/22/json?key="

UNIT = "unit=kmph"
ROAD_CLOSURE = "roadClosure=True"

PARAMS = "&" + UNIT + "&" + ROAD_CLOSURE

In [6]:
address = '30 Aldwych, London WC2B 4BG'
G = ox.graph_from_address(address, network_type="drive", dist=2000)

In [7]:
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
nodes, edges = ox.graph_to_gdfs(G)
#z = edges[edges['name'].apply(lambda x: isinstance(x, list))]
z = edges.loc[edges['name'] == 'Savoy Court']
coords = list(z.iloc[0].geometry.coords)
correct_coords = []
for i in coords:
    correct_coords.append((i[1], i[0]))

In [8]:
z

,,,osmid,name,highway,maxspeed,access,oneway,reversed,length,speed_kph,travel_time,geometry,lanes,ref,tunnel,junction,bridge,width,est_width,service
u,v,key,,,,,,,,,,,,,,,,,,,
107715,9515368,0,4253954,Savoy Court,residential,5 mph,NaN,False,True,47.141,8.0,21.2,"LINESTRING (-0.12109 51.51057, -0.12103 51.510...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9515368,107715,0,4253954,Savoy Court,residential,5 mph,NaN,False,False,47.141,8.0,21.2,"LINESTRING (-0.12075 51.51021, -0.12103 51.510...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
latitude = str(correct_coords[0][0])
longitude = str(correct_coords[0][1])
location = "point=" + latitude + ","+ longitude

request_url = BASE_URL + TRAFFIC_URL + API_KEY + "&" + location + PARAMS
# Get data
response = requests.get(request_url)

In [10]:
location

'point=51.5105733,-0.1210904'

In [11]:
data = response.json()['flowSegmentData']
traffic = pd.json_normalize(data)
traffic = traffic.rename(columns={'coordinates.coordinate': 'coordinates'})
traffic.currentSpeed[0]


19

In [12]:
traffic

,frc,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure,@version,coordinates
0,FRC4,19,19,738,738,1,False,traffic-service-flow 1.0.053,"[{'latitude': 51.51983791495153, 'longitude': ..."


In [13]:


coordinates = pd.json_normalize( traffic.iloc[0].coordinates)
#coordinates

#coordinates[round(coordinates.latitude, 7) == 34.0529563]
#coordinates.iloc[0].latitude
#coordinates.iloc[1]
coordinates.latitude = coordinates.latitude.round(7)
coordinates.longitude = coordinates.longitude.round(7)



In [14]:
geometry = LineString([(xy)  for xy in zip(coordinates['latitude'], coordinates['longitude'])])
#list(geometry.coords)


In [15]:
map = folium.Map(location=[correct_coords[0][0], correct_coords[0][1]],zoom_start=15)
# folium.PolyLine(list(geometry.coords)).add_to(map)
# folium.PolyLine(correct_coords).add_to(map)

In [16]:
for p in list(geometry.coords):
    folium.Marker(p, popup=p, icon=folium.Icon(color='red')).add_to(map)
    
for p in correct_coords:
    folium.Marker(p, popup=p).add_to(map)

In [17]:
map

In [18]:
correct_coords

[(51.5105733, -0.1210904), (51.5104995, -0.1210324), (51.5102051, -0.1207534)]